In [ ]:
import re
import pandas as pd
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from pyspark.sql import SparkSession

# ========================
# Initialize Azure Client
# ========================
endpoint = os.getenv("AZURE_FORM_RECOGNIZER_ENDPOINT", "your_endpoint_here")
key = os.getenv("AZURE_FORM_RECOGNIZER_KEY", "your_key_here")
pdf_path = os.getenv("PDF_PATH", "your_pdf_path_here")

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

# ========================
# Load PDF File
# ========================
with open(pdf_path, "rb") as f:
    poller = document_analysis_client.begin_analyze_document(
        "prebuilt-layout", document=f
    )
    result = poller.result()


In [ ]:
# ========================
# Process Page 1
# ========================
page1_data = []
date_pattern = r"\d{4}-\d{2}-\d{2}"
combined_pattern = r"*?(\d+).*?(\d{1,3}(?:,\d{3})*\.\d{2})\s*\.(\d{2})\."

if "result" in locals():
    for table in result.tables:
        if table.bounding_regions[0].page_number == 1:
            max_col = max(cell.column_index for cell in table.cells) + 1
            rows = {row_idx: [""] * max_col for row_idx in range(table.row_count)}

            for cell in table.cells:
                rows[cell.row_index][cell.column_index] = cell.content

            for row in rows.values():
                full_text = " ".join(row)
                date_match = re.search(date_pattern, full_text)
                current_date = date_match.group() if date_match else None

                combined_match = re.search(combined_pattern, full_text)
                if combined_match:
                    cars = int(combined_match.group(1))
                    main = combined_match.group(2).replace(",", "")
                    cents = combined_match.group(3)
                    total = float(main) + (float(cents) / 100)
                    page1_data.append([current_date, cars, total])

df_page1 = pd.DataFrame(page1_data, columns=["Date", "Total Number Of Cars", "Total"])
if not df_page1.empty:
    df_page1["Total"] = df_page1["Total"].apply(lambda x: f"{x:,.2f}")

df_page1


In [ ]:
# ========================
# Process Page 3
# ========================
page3_data = []
date_pattern = r"\b\d{4}-\d{2}-\d{2}\b"
date_found = None

if "result" in locals():
    for table in result.tables:
        if table.bounding_regions[0].page_number == 3:
            max_col = max(cell.column_index for cell in table.cells) + 1
            rows = {row_idx: [""] * max_col for row_idx in range(table.row_count)}

            for cell in table.cells:
                rows[cell.row_index][cell.column_index] = cell.content

            sorted_rows = [row[1] for row in sorted(rows.items())]

            for value in sorted_rows[0]:
                if isinstance(value, str) and re.search(date_pattern, value):
                    date_found = re.search(date_pattern, value).group()
                    break

            page3_data.extend(sorted_rows)

df_page3 = pd.DataFrame(page3_data)
if not df_page3.empty:
    df_page3.columns = df_page3.iloc[0]
    df_page3 = df_page3[1:].reset_index(drop=True)

df_page3["Date"] = date_found if date_found else "Missing Date"
if "Date" in df_page3.columns:
    col_to_move = df_page3.pop("Date")
    df_page3.insert(0, "Date", col_to_move)

if not df_page3.empty:
    df_page3 = df_page3.iloc[:-1]

columns_to_clean = ["entry", "exit", "total"]
for col in columns_to_clean:
    if col in df_page3.columns:
        df_page3[col] = df_page3[col].str.replace(r"\D", "", regex=True)
        df_page3[col] = df_page3[col].replace("", "0")
        df_page3[col] = df_page3[col].astype(int)

if "Peak percentage" in df_page3.columns:
    df_page3["Peak percentage"] = df_page3["Peak percentage"].str.replace(r"[^\d.%]", "", regex=True)

df_page3
